# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so don't need to load anything from disk.

In [1]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26

def idx_to_onehot(x, k=N+1):
  ones = torch.sparse.torch.eye(k).to(x.device)
  """ Converts the generated integers to one-hot vectors """
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])

class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=3, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class GRUMemory(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.gru = nn.GRU(input_size=N+1, hidden_size=hidden_size, batch_first=True)

        self.decoder = nn.Linear(hidden_size, N+1)

    def forward(self, x):

        output, _ = self.gru(x)

        logits = self.decoder(output)

        return logits

    @torch.no_grad()
    def test_run(self, s):

        device = next(self.parameters()).device
        indices = torch.tensor([ord(c) - ord('a') + 1 if 'a' <= c <= 'z' else 0 for c in s], dtype=torch.int64, device=device)
        one_hot = idx_to_onehot(indices).unsqueeze(0).to(device)

        output_logits = self(one_hot)

        predicted_indices = output_logits.argmax(dim=-1).squeeze(0)

        predicted_chars = ''.join(' ' if idx == 0 else chr(ord('a') + idx - 1) for idx in predicted_indices.cpu())

        return predicted_chars

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [11]:
def test_model(model):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  device = next(model.parameters()).device
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-DELAY], result[DELAY:]):
      correct += int(c1 == c2)
    total += len(s) - DELAY

  return correct / total

In [12]:
import time
import torch.optim as optim
from tqdm.notebook import tqdm
from torch import nn

HIDDEN_SIZE= 128 # TODO
BATCH_SIZE= 256 #TODO

def train_model(model, dataloader, num_epochs=5, lr=0.005):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Loss function: CrossEntropy since we predict N+1 classes
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)

            # Convert to one-hot encoding for inputs and integer targets
            inputs_one_hot = idx_to_onehot(inputs)

            optimizer.zero_grad()
            outputs = model(inputs_one_hot)

            # Reshape targets to match output shape
            loss = criterion(outputs.view(-1, N+1), targets.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(dataloader):.4f}")

model = GRUMemory(HIDDEN_SIZE)
dataloader = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE)

start_time = time.time()
train_model(model, dataloader)
end_time = time.time()

duration = end_time - start_time
accuracy = test_model(model)
print(f"Accuracy: {accuracy:.4f}")

assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

Epoch 1/5:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 1/5, Loss: 0.4980


Epoch 2/5:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 2/5, Loss: 0.0008


Epoch 3/5:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 3/5, Loss: 0.0002


Epoch 4/5:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 4/5, Loss: 0.0001


Epoch 5/5:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 5/5, Loss: 0.0001
Accuracy: 1.0000
tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay stays constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [13]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [17]:
class VariableDelayGRUMemory(nn.Module):

    def __init__(self, hidden_size, max_delay):
        super().__init__()
        self.hidden_size = hidden_size
        self.max_delay = max_delay

        self.gru = nn.GRU(input_size=N+2, hidden_size=hidden_size, batch_first=True)

        self.decoder = nn.Linear(hidden_size, N+1)

    def forward(self, x, delays):

        delays = delays.unsqueeze(1).expand(-1, x.shape[1]).unsqueeze(-1)
        x = torch.cat([x, delays], dim=-1)

        output, _ = self.gru(x)

        logits = self.decoder(output)

        return logits

    @torch.no_grad()
    def test_run(self, s, delay):

        device = next(self.parameters()).device

        indices = torch.tensor(
            [ord(c) - ord('a') + 1 if 'a' <= c <= 'z' else 0 for c in s],
            dtype=torch.int64,
            device=device
        )

        one_hot = idx_to_onehot(indices).unsqueeze(0)

        delay_tensor = torch.tensor([delay], dtype=torch.float32, device=device)

        output_logits = self(one_hot, delay_tensor)

        predicted_indices = output_logits.argmax(dim=-1).squeeze(0)

        predicted_chars = ''.join(
            ' ' if idx == 0 else chr(ord('a') + idx - 1) for idx in predicted_indices.cpu()
        )

        return predicted_chars

In [18]:
def test_variable_delay_model(model, max_delay=12):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  delay = random.randint(0, max_delay)

  total = 0
  correct = 0
  for i in range(1000):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s, delay)
    for c1, c2 in zip(s[:-delay], result[delay:]):
      correct += int(c1 == c2)
    total += len(s) - delay

  return correct / total

### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [19]:
import time

MAX_DELAY = 12
SEQ_LENGTH = 20


from tqdm.notebook import tqdm
from torch import nn

HIDDEN_SIZE= 128 # TODO
BATCH_SIZE= 256 #TODO

def train_variable_delay_model(model, dataloader, num_epochs=5, lr=0.005):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Loss function: CrossEntropy for classification
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            inputs, delays, targets = batch
            inputs, delays, targets = inputs.to(device), delays.to(device), targets.to(device)

            inputs_one_hot = idx_to_onehot(inputs)

            optimizer.zero_grad()
            outputs = model(inputs_one_hot, delays)

            loss = criterion(outputs.view(-1, N+1), targets.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(dataloader):.4f}")

model = VariableDelayGRUMemory(hidden_size=HIDDEN_SIZE, max_delay=MAX_DELAY)
ds = VariableDelayEchoDataset(max_delay=MAX_DELAY, size=600000, seq_length=SEQ_LENGTH)
dataloader = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE)

start_time = time.time()
train_variable_delay_model(model, dataloader)
end_time = time.time()

assert end_time - start_time < 600, 'executing took longer than 10 mins'
accuracy = test_variable_delay_model(model, max_delay=MAX_DELAY)
print(accuracy)
assert accuracy > 0.95, 'accuracy is too low'
print('tests passed')

Epoch 1/5:   0%|          | 0/2344 [00:00<?, ?it/s]

Epoch 1/5, Loss: 0.5605


Epoch 2/5:   0%|          | 0/2344 [00:00<?, ?it/s]

Epoch 2/5, Loss: 0.1481


Epoch 3/5:   0%|          | 0/2344 [00:00<?, ?it/s]

Epoch 3/5, Loss: 0.0767


Epoch 4/5:   0%|          | 0/2344 [00:00<?, ?it/s]

Epoch 4/5, Loss: 0.0482


Epoch 5/5:   0%|          | 0/2344 [00:00<?, ?it/s]

Epoch 5/5, Loss: 0.0426
0.997995858106754
tests passed
